문제1 10점   
입력 데이터:  
다운로드: https://github.com/nongaussian/class-2020-spark/blob/master/Week%207/input.zip  
설명 - 영화DB사이트 (MovieLens)에서 수집된 데이터로서 u.data에는 사용자가 영화에 매긴 평점이 담겨있고  
u.item에는 영화마다 제목, 카테고리 등의 정보가 담겨 있다.  
  
입력파일 1 - u.data  

* 라인마다 사용자의 영화평점이 기록되어 있으며 4개의 속성 "user id \t item id \t rating \t timestamp"가 탭으로 구분되어 작성되어 있다.  
* (즉, split("\t") 로 자를 수 있음)  모두 integer값이다.  
* 한 사람이 여러개의 영화에 평점을 매길 수 있으므로 같은 user id로 시작하는 라인이 복수번 나타날수 있다.  
* item id는 아래 u.item파일에 있는 영화 아이디 (movie id)를 가리킨다 (즉, foreign key)  
* rating은 1에서 5까지 범위이다.  
  
입력파일 2 - u.item  
  
* 라인마다 영화 정보가 기록되어 있으며 "movie id | movie title | release date | video release date |
* IMDb URL | unknown | Action | Adventure | Animation |
* Children's | Comedy | Crime | Documentary | Drama | Fantasy |
* Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
* Thriller | War | Western" 의 속성 들이 파이프라인(|)으로 구분되어 작성되어 있다.
* 그러나 아래 문제를 보면 맨 앞의 두개 속성, movie id와 movie title 만 사용한다.
* movie id는 이 파일 안에서 한번만 나타난다 (즉, primary key)
  
문제.  
5점을 받은 영화 중에서 알파벳 'T'로 시작하는 영화의 개수는?  
'T'는 대문자  
주의: 5점을 받은 영화중 알파벳 'T'로 시작하는 영화를 중복해서 세지 않도록!  
제한조건: 마지막 action으로 답이 출력되도록 코드를 작성하시오.   
즉, 최종 답을 계산하기 위해 pyspark action이 아닌 python 코드를 사용하지 마시오.  


In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext("local", "transformation 3")

In [4]:
# 공백으로 구분하여 item id (movie id)와 rating만 슬라이싱하여 data로 추출한다 
data = sc.textFile('u.data').map(lambda x: x.split("\t")[1:3])
print(data.distinct())
# data.collect()
# [['242', '3'],
#  ['302', '3'],
#  ['377', '1'],
#  ['51', '2'],
#  ['346', '1'],
#  ['474', '4'],,,,]

PythonRDD[6] at RDD at PythonRDD.scala:53


In [6]:
# groupByKey()이용할 수 있지만 cgroup이용하면 동일한 효과를 볼 수 있다
data.groupByKey().mapValues(list).collect()
# [('242',
#   ['3',
#    '3',
#    '5',
#    '3',
#    '5',
#    '4',
#    '5',

In [39]:
# 중복확인 # 중복확인되나  cgroup에서 정리될 것이므로 문제되지 않음
# data.map(lambda x: (x[0], 1)).collect()
# data.map(lambda x: (x[0], 1)).reduceByKey(lambda a, b: a + b).filter(lambda y: y[1] != 1).count() # 1541
data.map(lambda x: (x[0], 1)).reduceByKey(lambda a, b: a + b).filter(lambda y: y[1] == 1).count() # 141

141

In [7]:
# |로 구분하여  movie id 와 movie title만 슬라이싱하여 item으로 추출한다 # 중복값 제거
# item = sc.textFile('u.item').map(lambda x: x.split("|")[0:2]).distinct() # error
item = sc.textFile('u.item').map(lambda x: x.split("|")[0:2])
# item.collect()
# [['1', 'Toy Story (1995)'],
#  ['2', 'GoldenEye (1995)'],
#  ['3', 'Four Rooms (1995)'],
#  ['4', 'Get Shorty (1995)'],
#  ['5', 'Copycat (1995)'],,,]

In [8]:
# 중복확인 없음
# item.map(lambda x: (x[0], 1)).collect()
item.map(lambda x: (x[0], 1)).reduceByKey(lambda a, b: a + b).filter(lambda y: y[1] != 1).count() # 0
# item.map(lambda x: (x[0], 1)).reduceByKey(lambda a, b: a + b).filter(lambda y: y[1] == 1).count() # 1682

0

In [11]:
# 중복확인 18개에 대해 중복제거되어야 함
# item.map(lambda x: (x[1], 1)).collect()
# item.map(lambda x: (x[1], 1)).reduceByKey(lambda a, b: a + b).filter(lambda y: y[1] != 1).count() # 18
# item.map(lambda x: (x[1], 1)).reduceByKey(lambda a, b: a + b).filter(lambda y: y[1] == 1).count() # 1646

item.map(lambda x: (x[1], 1)).reduceByKey(lambda a, b: a + b).filter(lambda x: x[1] != 1).collect()
# [('Chasing Amy (1997)', 2),
#  ('Kull the Conqueror (1997)', 2),
#  ("Ulee's Gold (1997)", 2),
#  ('Fly Away Home (1996)', 2),
#  ('Ice Storm, The (1997)', 2),
#  ('Deceiver (1997)', 2),
#  ('Desperate Measures (1998)', 2),
#  ('Body Snatchers (1993)', 2),
#  ('Substance of Fire, The (1996)', 2),
#  ('Money Talks (1997)', 2),
#  ('That Darn Cat! (1997)', 2),
#  ('Hugo Pool (1997)', 2),
#  ('Chairman of the Board (1998)', 2),
#  ('Designated Mourner, The (1997)', 2),
#  ('Hurricane Streets (1998)', 2),
#  ('Sliding Doors (1998)', 2),
#  ('Nightwatch (1997)', 2),
#  ('Butcher Boy, The (1998)', 2)]

In [22]:
# item.map(lambda x: (x[1], 1)).reduceByKey(lambda a, b: a + b).filter(lambda x: x[1] != 1 and x[0][:1] == 'T').collect()
# [('That Darn Cat! (1997)', 2)]

item.map(lambda x: (x[1], 1)).reduceByKey(lambda a, b: a + b).filter(lambda x: x[1] != 1 and x[0][:1] == 'T').keys().collect()
# ['That Darn Cat! (1997)']

# 중복된 영화(5점이상, 첫글자 'T')
item.map(lambda x: (x[1], 1)).reduceByKey(lambda a, b: a + b).filter(lambda x: x[1] != 1 and x[0][:1] == 'T').keys().distinct()

# 중복된 영화(5점이상, 첫글자 'T') 중복횟수
duplicate_count = item.map(lambda x: (x[1], 1)).reduceByKey(lambda a, b: a + b).filter(lambda x: x[1] != 1 and x[0][:1] == 'T').values()
duplicate_count.collect()[0]  # 2

2

In [24]:
# cogroup 이용
# groupByKey()는 동일 rdd내에서, cogroup은 다른 2개의 RDD의 key값을 중심으로 (key, tuple(lterable, lterable,,))
rdd = item.cogroup(data).mapValues(lambda t: ( list(t[0]), list(t[1]) ))
# rdd.collect()
# [('1',
#   (['Toy Story (1995)'],
#    ['4',
#     '5',
#     '4',
#     '4',
#     '3',
#     '4',
#     '4',
#     '3',
#     '5',
#     '3',
#     '5',
#     '3',
#     '4',

In [15]:
# move id만
rdd.map(lambda x: x[0]).collect()
# ['1',
#  '4',
#  '8',
#  '9',
#  '10',
#  '12',
#  '14',
#  '16',
#  '17',
#  '19',
#  '20',
#  '21',

In [37]:
rdd.filter(lambda x : x[1][0][0] == 'That Darn Cat! (1997)').collect()

[('878',
  (['That Darn Cat! (1997)'],
   ['5',
    '2',
    '1',
    '1',
    '1',
    '2',
    '1',
    '1',
    '4',
    '2',
    '2',
    '2',
    '1',
    '2',
    '3',
    '1',
    '5',
    '5',
    '3',
    '2',
    '3',
    '4',
    '1',
    '4',
    '2',
    '5',
    '3',
    '1',
    '4',
    '2',
    '1',
    '4',
    '3'])),
 ('1003',
  (['That Darn Cat! (1997)'], ['2', '2', '3', '1', '4', '1', '3', '2']))]

In [16]:
# rdd중에서 중복된 값이 있는지 확인 # 없다
rdd.map(lambda x: (x[0], 1)).reduceByKey(lambda a, b: a + b).filter(lambda y: y[1] != 1).count()
# 0

0

In [18]:
# rating, movie title
rdd.map(lambda x: x[1]).collect()
# [(['Toy Story (1995)'],
#   ['4',
#    '5',
#    '4',
#    '4',
#    '3',
#    '4',
#    '4',
#    '3',

In [21]:
rdd.map(lambda x: x[1][0]).collect()
# [['Toy Story (1995)'],
#  ['Get Shorty (1995)'],
#  ['Babe (1995)'],
#  ['Dead Man Walking (1995)'],
#  ['Richard III (1995)'],
#  ['Usual Suspects, The (1995)'],
#  ['Postino, Il (1994)'],

In [23]:
rdd.map(lambda x: x[1][1]).collect() 
# [['4',
#   '5',
#   '4',
#   '4',
#   '3',
#   '4',
#   '4',
#   '3',

In [25]:
# rdd.map(lambda x : x[1][0].contains['5']).collect() # error
# rdd.map(lambda x: x[1][0]).filter(lambda y : y =='5' ).collect() # error
# rdd.map(lambda x: x[1][0]).filter(lambda y : y.contains['5'] ).collect() # error
# rdd.filter(lambda x : x[1][0].contains['5'] ).collect() # error
# rdd.filter(lambda x : x[1][0] in '5' ).collect() # error
# rdd.filter(lambda x : x[1][1][0:2] in 'T' ).collect() # error
# rdd.filter(lambda x : '5' in x[1][0] and 'T' in x[1][1]).collect() # []

rdd.filter(lambda x : '5' in x[1][1]).collect() 
# [('1',
#   (['Toy Story (1995)'],
#    ['4',
#     '5',
#     '4',
#     '4',
#     '3',
#     '4',


In [28]:
rdd.map(lambda x : x[1][0][0][0:1]).collect() 
# ['T',
#  'G',
#  'B',
#  'D',
#  'R',
#  'U',
#  'P',
#  'F',
 

In [30]:
rdd.filter(lambda x : '5' in x[1][1] and x[1][0][0][0:1] == 'T').collect()
# [('1',
#   (['Toy Story (1995)'],
#    ['4',
#     '5',
#     '4',
#     '4',
#     '3',
#     '4',
#     '4',

In [31]:
rdd.filter(lambda x : '5' in x[1][1] and x[1][0][0][0:1] == 'T').collect()
# [('1',
#   (['Toy Story (1995)'],
#    ['4',
#     '5',
#     '4',
#     '4',
#     '3',
#     '4',
#     '4',
#     '3',
#     '5',

rdd.filter(lambda x : '5' in x[1][1] and x[1][0][0][0:1] == 'T').map(lambda x: x[1][0][0]).collect()
# ['Toy Story (1995)',
#  'To Wong Foo, Thanks for Everything! Julie Newmar (1995)',
#  'Three Colors: Blue (1993)',
#  'Terminator 2: Judgment Day (1991)',
#  'Truth About Cats & Dogs, The (1996)',
#  'Top Gun (1986)',

In [35]:
#  (5점이상, 첫글자 'T')인 movie title 추출 
# 그런데 중복된 'That Darn Cat! (1997)'이 중복이 나타나지 않음
rdd.filter(lambda x : '5' in x[1][1] and x[1][0][0][0:1] == 'T').map(lambda x: x[1][0][0]).map(lambda x: x.strip()).collect()
# ['Toy Story (1995)',
#  'To Wong Foo, Thanks for Everything! Julie Newmar (1995)',
#  'Three Colors: Blue (1993)',
#  'Terminator 2: Judgment Day (1991)',
#  'Truth About Cats & Dogs, The (1996)',
#  'Top Gun (1986)',
#  'This Is Spinal Tap (1984)',
#  'Tin Cup (1996)',
#  'True Lies (1994)',

['Toy Story (1995)',
 'To Wong Foo, Thanks for Everything! Julie Newmar (1995)',
 'Three Colors: Blue (1993)',
 'Terminator 2: Judgment Day (1991)',
 'Truth About Cats & Dogs, The (1996)',
 'Top Gun (1986)',
 'This Is Spinal Tap (1984)',
 'Tin Cup (1996)',
 'True Lies (1994)',
 'Tales from the Crypt Presents: Bordello of Blood (1996)',
 'Transformers: The Movie, The (1986)',
 'Tombstone (1993)',
 'To Catch a Thief (1955)',
 'Third Man, The (1949)',
 'Treasure of the Sierra Madre, The (1948)',
 'Thin Blue Line, The (1988)',
 'Tin Men (1987)',
 'To Die For (1995)',
 'Threesome (1994)',
 'Tommy Boy (1995)',
 'Timecop (1994)',
 'To Gillian on Her 37th Birthday (1996)',
 'Telling Lies in America (1997)',
 'Tango Lesson, The (1997)',
 'Trigger Effect, The (1996)',
 'Turbulence (1997)',
 'Trees Lounge (1996)',
 'Tie Me Up! Tie Me Down! (1990)',
 'Two if by Sea (1996)',
 'Thin Line Between Love and Hate, A (1996)',
 'Tank Girl (1995)',
 'That Old Feeling (1997)',
 'To Be or Not to Be (1942)',


In [33]:
#  (5점이상, 첫글자 'T')인 movie title 추출해서 중복제거 
rdd.filter(lambda x : '5' in x[1][1] and x[1][0][0][0:1] == 'T').map(lambda x: x[1][0][0]).map(lambda x: x.strip()).distinct().count()

75

In [34]:
rdd.filter(lambda x : '5' in x[1][1] and x[1][0][0][0:1] == 'T').map(lambda x: x[1][0][0]).map(lambda x: x.strip()).map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b).filter(lambda x: x[1] !=1).collect()

[]

In [31]:
type(rdd.filter(lambda x : '5' in x[1][1] and x[1][0][0][0:1] == 'T').collect()) # list
len(rdd.filter(lambda x : '5' in x[1][1] and x[1][0][0][0:1] == 'T').collect())

75

문제 2 10점  
문제.  
사용자들에게 1점도 받고 5점도 받은 적이 있는 영화의 제목의 길이 중 가장 긴 영화제목의 길이는?  
  
동일 영화가 어떤 사용자에게는 1점, 어떤 사용자에게는 5점을 받은 경우를 말함  
힌트: cogroup을 쓸 수도 있고 groupByKey를 쓸 수도 있음.  
제한조건: 마지막 action으로 답이 출력되도록 코드를 작성하시오.   
즉, 최종 답을 계산하기 위해 pyspark action이 아닌 python 코드를 사용하지 마시오.  

In [38]:
# 사용자들에게 1점도 받고 5점도 받은 적이 있는 영화
# rdd.filter(lambda x : ('5' in x[1][1]) and ('1' in x[1][1])).collect()
# (['Gold Diggers: The Secret of Bear Mountain (1995)'],
#    ['2', '3', '3', '1', '4', '2', '5', '4', '4', '3'])),
rdd_filtered_rating  = rdd.filter(lambda x : ('5' in x[1][1]) and ('1' in x[1][1]))

In [46]:
# 영화제목(movie title) 추출
# rdd_filtered_rating.map(lambda x: x[1][0][0]).collect()
# ['Toy Story (1995)',
#  'Get Shorty (1995)',
#  'Babe (1995)',
#  'Dead Man Walking (1995)',
#  'Richard III (1995)',
#  'Usual Suspects, The (1995)',
#  'Postino, Il (1994)',
#  'French Twist (Gazon maudit) (1995)',
#  'From Dusk Till Dawn (1996)'

# 영화제목 길이 계산
# rdd_filtered_rating.map(lambda x: x[1][0][0]).map(lambda x: len(x)).collect()
# [16,
#  17,
#  11,
#  23,
#  18,
#  26,
#  18,
#  34,
#  26,

In [50]:
# 영화제목 길이로 sortBy(내림차순)해서 처음을 추출, 길이
len(rdd_filtered_rating.map(lambda x: x[1][0][0]).sortBy(lambda x: len(x), False).collect()[0])

81